In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random


In [2]:
fake = Faker("en_IN")
np.random.seed(42)
random.seed(42)


In [3]:
NUM_CUSTOMERS = 30000

customers = []
for i in range(1, NUM_CUSTOMERS + 1):
    customers.append({
        "customer_id": i,
        "customer_age": np.random.randint(22, 65),
        "gender": random.choice(["M", "F"]),
        "employment_type": random.choice(["Salaried", "Self-Employed"]),
        "monthly_income": np.random.randint(15000, 120000),
        "credit_score": np.random.randint(550, 850),
        "city": fake.city(),
        "state": fake.state(),
        "region": random.choice(["North", "South", "East", "West"]),
        "customer_segment": random.choice(["Prime", "Near-Prime", "Sub-Prime"])
    })

dim_customer = pd.DataFrame(customers)


In [4]:
branches = []
for i in range(1, 51):
    branches.append({
        "branch_id": i,
        "branch_name": f"Branch_{i}",
        "city": fake.city(),
        "state": fake.state(),
        "region": random.choice(["North", "South", "East", "West"]),
        "branch_type": random.choice(["Urban", "Semi-Urban", "Rural"])
    })

dim_branch = pd.DataFrame(branches)


In [5]:
agents = []
for i in range(1, 201):
    agents.append({
        "agent_id": i,
        "agent_name": fake.name(),
        "branch_id": random.randint(1, 50),
        "role": random.choice(["Telecaller", "Field"]),
        "experience_years": np.random.randint(1, 10),
        "agent_status": random.choice(["Active", "Active", "Inactive"])
    })

dim_agent = pd.DataFrame(agents)


In [6]:
NUM_LOANS = 5000
start_date = datetime(2020, 1, 1)

loans = []
for i in range(1, NUM_LOANS + 1):
    loan_amount = np.random.randint(50000, 500000)
    outstanding = loan_amount * np.random.uniform(0.1, 1.0)
    dpd = random.choice([0, 0, 5, 15, 30, 45, 60, 90, 120])

    loans.append({
        "loan_account_id": i,
        "customer_id": random.randint(1, NUM_CUSTOMERS),
        "product_type": random.choice(["Personal Loan", "Gold Loan", "MSME Loan"]),
        "disbursement_date": start_date + timedelta(days=random.randint(0, 1200)),
        "loan_amount": loan_amount,
        "outstanding_principal": round(outstanding, 2),
        "emi_amount": round(loan_amount / np.random.randint(12, 48), 2),
        "tenure_months": random.choice([12, 24, 36, 48]),
        "interest_rate": round(np.random.uniform(10, 24), 2),
        "current_dpd": dpd,
        "max_dpd_ever": max(dpd, random.choice([30, 60, 90])),
        "delinquency_bucket": (
            "0-30" if dpd <= 30 else
            "31-60" if dpd <= 60 else
            "61-90" if dpd <= 90 else "90+"
        ),
        "loan_status": random.choice(["Active", "Active", "Closed", "Write-off"])
    })

fact_loan_account = pd.DataFrame(loans)


In [7]:
repayments = []

for _, loan in fact_loan_account.iterrows():
    emi_count = random.randint(6, 24)
    for i in range(emi_count):
        due_date = loan["disbursement_date"] + timedelta(days=30 * i)
        delay = random.choice([0, 0, 2, 5, 10, 30, 60])

        repayments.append({
            "repayment_id": len(repayments) + 1,
            "loan_account_id": loan["loan_account_id"],
            "due_date": due_date,
            "payment_date": due_date + timedelta(days=delay) if delay < 60 else None,
            "amount_due": loan["emi_amount"],
            "amount_paid": loan["emi_amount"] if delay < 60 else 0,
            "days_delay": delay,
            "payment_status": (
                "On-time" if delay == 0 else
                "Late" if delay <= 30 else "Missed"
            ),
            "payment_channel": random.choice(["UPI", "Cash", "Bank Transfer"])
        })

fact_repayment = pd.DataFrame(repayments)


In [8]:
snapshots = []

for _, loan in fact_loan_account.iterrows():
    dpd = loan["current_dpd"]
    for m in range(6):
        snapshots.append({
            "snapshot_date": datetime.today() - timedelta(days=30 * m),
            "loan_account_id": loan["loan_account_id"],
            "dpd": max(0, dpd - m * random.randint(5, 15)),
            "dpd_bucket": loan["delinquency_bucket"],
            "outstanding_amount": round(loan["outstanding_principal"] * np.random.uniform(0.8, 1.0), 2)
        })

fact_dpd_snapshot = pd.DataFrame(snapshots)


In [9]:
contacts = []

for _, loan in fact_loan_account.iterrows():
    attempts = random.randint(1, 6)
    for _ in range(attempts):
        contacts.append({
            "contact_id": len(contacts) + 1,
            "loan_account_id": loan["loan_account_id"],
            "agent_id": random.randint(1, 200),
            "contact_date": datetime.today() - timedelta(days=random.randint(1, 90)),
            "contact_type": random.choice(["Call", "Visit", "SMS"]),
            "contact_outcome": random.choice(["Connected", "No Answer", "Switched Off"]),
            "right_party_contact": random.choice([True, False]),
            "remarks": random.choice(["PTP taken", "Call back", "Refused"])
        })

fact_collection_contact = pd.DataFrame(contacts)


In [10]:
ptps = []

for _, contact in fact_collection_contact.sample(3000).iterrows():
    promised_date = contact["contact_date"] + timedelta(days=random.randint(1, 15))
    kept = random.choice([True, False, False])

    ptps.append({
        "ptp_id": len(ptps) + 1,
        "loan_account_id": contact["loan_account_id"],
        "agent_id": contact["agent_id"],
        "ptp_date": contact["contact_date"],
        "promised_amount": random.randint(2000, 15000),
        "promised_payment_date": promised_date,
        "actual_payment_date": promised_date if kept else None,
        "ptp_status": "Kept" if kept else "Broken"
    })

fact_ptp = pd.DataFrame(ptps)


In [11]:
resolutions = []

for _, loan in fact_loan_account.sample(1500).iterrows():
    resolutions.append({
        "resolution_id": len(resolutions) + 1,
        "loan_account_id": loan["loan_account_id"],
        "resolution_type": random.choice(["Regularised", "Settlement", "Write-off"]),
        "resolution_date": datetime.today() - timedelta(days=random.randint(30, 180)),
        "recovered_amount": round(loan["outstanding_principal"] * np.random.uniform(0.3, 1.0), 2),
        "resolution_tat_days": random.randint(15, 120)
    })

fact_resolution = pd.DataFrame(resolutions)


In [12]:
dim_customer.to_csv("dim_customer.csv", index=False)
dim_branch.to_csv("dim_branch.csv", index=False)
dim_agent.to_csv("dim_agent.csv", index=False)
fact_loan_account.to_csv("fact_loan_account.csv", index=False)
fact_repayment.to_csv("fact_repayment.csv", index=False)
fact_dpd_snapshot.to_csv("fact_dpd_snapshot.csv", index=False)
fact_collection_contact.to_csv("fact_collection_contact.csv", index=False)
fact_ptp.to_csv("fact_ptp.csv", index=False)
fact_resolution.to_csv("fact_resolution.csv", index=False)
